In [1]:
import ase
import ase.io
import jax
import jax.numpy as jnp
import jraph
import ml_collections
import numpy as np
import sys

sys.path.append('..')
from analyses import analysis
from symphony import train
from symphony.data import input_pipeline_tf

In [2]:
workdir = '/data/NFS/potato/songk/spherical-harmonic-net/workdirs/silica-mini-nequip-jan3'
rng = jax.random.PRNGKey(0)

In [3]:
config, best_state, best_eval_state, metrics_for_best_state = analysis.load_from_workdir(workdir)
config = ml_collections.FrozenConfigDict(config)
datasets = input_pipeline_tf.get_datasets(rng, config)

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 5462683543248128186
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 5462683543248128186
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing te

In [4]:
metrics_for_best_state

{'train_eval': {'total_loss': array(8.642514, dtype=float32),
  'focus_and_atom_type_loss': array(3.6723154, dtype=float32),
  'position_loss': array(4.970197, dtype=float32)},
 'val_eval': {'total_loss': array(9.117983, dtype=float32),
  'focus_and_atom_type_loss': array(3.8686967, dtype=float32),
  'position_loss': array(5.249269, dtype=float32)},
 'test_eval': {'total_loss': array(14.887489, dtype=float32),
  'focus_and_atom_type_loss': array(3.50409, dtype=float32),
  'position_loss': array(11.383399, dtype=float32)}}

In [84]:
frag_batch = next(datasets['train'].as_numpy_iterator())
unbatched_frags = jraph.unbatch(frag_batch)

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 17407362735305946880
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 7041437071164992347
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing t

In [85]:
frag = unbatched_frags[0]

In [7]:
step_rng, rng = jax.random.split(rng)
step_rngs = jax.random.split(step_rng, jax.local_device_count())
preds = train.get_predictions(best_eval_state, frag, rng)

In [80]:
frag_ase = ase.Atoms(
    positions = frag.nodes.positions,
    numbers = np.vectorize(lambda x: 14 if x else 8)(frag.nodes.species),
    cell = frag.globals.cell[0],
    pbc = np.array([1, 1, 1]),
)
ase.io.write('frag.cif', frag_ase)

In [87]:
frag.receivers[frag.senders == 9]

Array([8], dtype=int32)

In [67]:
frag.globals.stop, preds.globals.stop

(Array([False], dtype=bool), Array([False], dtype=bool))

In [50]:
frag.nodes.focus_and_target_species_probs

Array([[1., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)

In [10]:
preds.nodes.focus_and_target_species_probs  # what's going on here

Array([[2.70676780e-02, 2.32231570e-03],
       [4.57778908e-02, 4.71724290e-03],
       [3.55939195e-02, 3.35815060e-03],
       [4.22604004e-04, 3.82231483e-05],
       [4.57859226e-02, 4.71852068e-03],
       [4.57778908e-02, 4.71724290e-03],
       [6.71012886e-03, 4.45996271e-03],
       [1.23545192e-02, 6.50395313e-03],
       [2.54839510e-02, 8.29199143e-03],
       [2.54839510e-02, 8.29199143e-03],
       [2.55051441e-02, 8.28829035e-03],
       [2.20253356e-02, 1.18094990e-02],
       [2.20058113e-02, 1.18096657e-02],
       [2.49284282e-02, 8.10649525e-03],
       [2.13306230e-02, 1.24294451e-02],
       [2.52077542e-02, 8.20001960e-03],
       [1.00042932e-02, 9.70991794e-03],
       [6.04603114e-03, 5.08677680e-03],
       [2.69503854e-02, 2.32886942e-03],
       [2.89604701e-02, 3.42380279e-03],
       [2.71169096e-02, 2.36592395e-03],
       [2.06524245e-02, 9.14766628e-04],
       [2.89664436e-02, 3.42479837e-03],
       [6.13711134e-04, 1.27833628e-04],
       [4.356529